# Using the BRepNet input features as a face and edge similarity metric
This example notebook shows how the BRepNet input features can be used as a similarity metric for finding similar faces

In [1]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
import os
if os.path.isfile('../models/brepnet.py'):
    # We are in the notebooks directory.  Change to the root
    os.chdir('../')

In [2]:
from pathlib import Path

# This viewer allows you to visualize the results
from visualization.jupyter_segmentation_viewer import JupyterSegmentationViewer

# Useful functions for loading data
import utils.data_utils as data_utils

In [3]:
# Here is the path to some example step files for us to convert
step_folder = Path("./example_files/step_examples")


# We will also need to know the feature standardization for the dataset used to train the model
# This is found in the dataset file created by pipeline/build_dataset_file.py or pipeline/quickstart.py
feature_standardization_path = Path("./example_files/feature_standardization/s2.0.0_step_all_features.json")
standardization_data = data_utils.load_json_data(feature_standardization_path)
feature_standardization = standardization_data["feature_standardization"]

# This function to standardize features is similar to what we do in the dataloader
def standardize_features(feature_tensor, stats):
    num_features = len(stats)
    assert feature_tensor.shape[1] == num_features
    means = np.zeros(num_features)
    sds = np.zeros(num_features)
    eps = 1e-7
    for index, s in enumerate(stats):
        assert s["standard_deviation"] > eps, "Feature has zero standard deviation"
        means[index] = s["mean"]
        sds[index] = s["standard_deviation"]

    # We need to broadcast means and sds over the number of entities
    means_x = np.expand_dims(means, axis=0)
    sds_x = np.expand_dims(sds, axis=0)
    feature_tensor_zero_mean = feature_tensor - means_x
    feature_tensor_standadized = feature_tensor_zero_mean / sds_x

    # Convert the tensors to floats after standardization 
    return feature_tensor_standadized
    
def standarize_data(data, feature_standardization):
    data["face_features"] = standardize_features(data["face_features"], feature_standardization["face_features"])
    data["edge_features"] = standardize_features(data["edge_features"], feature_standardization["edge_features"])
    data["coedge_features"] = standardize_features(data["coedge_features"], feature_standardization["coedge_features"])
    return data
    
def find_faces_to_edges(coedge_to_face, coedge_to_edge):
    faces_to_edges_dict = {}
    for coedge_index in range(coedge_to_face.shape[0]):
        edge = coedge_to_edge[coedge_index]
        face = coedge_to_face[coedge_index]
        if not face in faces_to_edges_dict:
            faces_to_edges_dict[face] = set()
        faces_to_edges_dict[face].add(edge)
    faces_to_edges = []
    for i in range(len(faces_to_edges_dict)):
        assert i in faces_to_edges_dict
        faces_to_edges.append(faces_to_edges_dict[i])
    return faces_to_edges

# This function pools edges features onto all faces which
# are adjacent to that edge
def pool_edge_data_onto_faces(data):
    face_features = data["face_features"]
    edge_features = data["edge_features"]
    coedge_to_face = data["coedge_to_face"] 
    coedge_to_edge = data["coedge_to_edge"]
    for edge in coedge_to_edge:
        assert edge < edge_features.shape[0]
    faces_to_edges = find_faces_to_edges(coedge_to_face, coedge_to_edge)
    face_edge_features = []
    for face_edge_set in faces_to_edges:
        edge_features_for_face  = []
        for edge in face_edge_set:
            edge_features_for_face.append(edge_features[edge])
        pooled_edge_features = np.max(np.stack(edge_features_for_face), axis = 0)
        face_edge_features.append(pooled_edge_features)
    assert len(face_edge_features) == face_features.shape[0]
    face_edge_features = np.stack(face_edge_features)
    return np.concatenate([face_features, face_edge_features], axis = 1)

Choose the model you would like to use as the query

In [4]:
step_file_stems = [ f.stem for f in step_folder.glob("*.stp")]
print(f"We found {len(step_file_stems)} example files")

We found 25 example files


In [5]:
query_index = 17

Now you will need to select some faces on the solid to continue.  
Double click on each face to select it

In [6]:
file_stem = step_file_stems[query_index]
print(f"Viewing example {file_stem}")
viewer = JupyterSegmentationViewer(file_stem, step_folder, seg_folder=step_folder)
viewer.view_solid()

Viewing example 21242_6c2af7c2_7


In [8]:
assert len(viewer.selection_list) > 0, "Please select some faces on the solid"

In [9]:
# Load raw feature data
npz_folder = step_folder / "temp_working"
npz_pathname = npz_folder / (file_stem + ".npz")
data = data_utils.load_npz_data(npz_pathname)
data = standarize_data(data, feature_standardization)
face_features = data["face_features"]
edge_features = data["edge_features"]
pooled_face_edge_features = pool_edge_data_onto_faces(data)

assert pooled_face_edge_features.shape[0] == len(viewer.entity_mapper.face_map), "Embedding size doesn't match solid"
selected_faces_features = pooled_face_edge_features[viewer.selection_list]

In [10]:
all_files = list(npz_folder.glob("*.npz"))

min_dists_for_each_face = []
sum_min_dists_for_each_solid = []
for npz_file in all_files:
        
    # Load the input features
    data = data_utils.load_npz_data(npz_file)
    data = standarize_data(data, feature_standardization)
    pooled_face_edge_features = pool_edge_data_onto_faces(data)
        
    min_dists = []
    min_dists_for_each_face_in_this_solid = []
    for selected_face_features in selected_faces_features:
        vec = pooled_face_edge_features-selected_face_features
        dist = np.linalg.norm(vec, axis=1)
        
        # Here we compute the distance from a given query face
        # to each face in the solid
        min_dists_for_each_face_in_this_solid.append(dist)
        
        # Then is metric is the distance from that given query face to
        # the most similar face in the solid
        min_dists.append(np.min(dist))
    
    # Now we need some way to say which solids is the best match over all
    # We do this by summing the minimum distances to all the faces 
    sum_dists = np.sum(np.stack(min_dists), axis=0)
    sum_min_dists_for_each_solid.append(sum_dists)
    
    # For display we also keep track of the min distance to each face
    min_dists_for_each_face_in_this_solid = np.min(np.stack(min_dists_for_each_face_in_this_solid), axis=0)
    min_dists_for_each_face.append(min_dists_for_each_face_in_this_solid)
    
sum_min_dists_for_each_solid = np.array(sum_min_dists_for_each_solid)  

In [11]:
# Now we will search for the top k best matches
k = 5
indices_of_smallest = np.argpartition(sum_min_dists_for_each_solid, kth=range(k))[:k]

# We want to show how close each face in each solid is
# To do this we want some kind of range which is computed here
all_dists_top_k = []
for index in indices_of_smallest:
    all_dists_top_k.append(min_dists_for_each_face[index])
all_dists_top_k = np.concatenate(all_dists_top_k)
interval = [all_dists_top_k.min(), all_dists_top_k.max()]
print(f"Interval {interval}")

Interval [0.0, 21.304689088927404]


In [12]:
# Now as a sanity check we always expect to find the query as the best 
# match.  The other matching faces (red) should have a similar shape
for i, index in enumerate(indices_of_smallest):
    print(index)
    close_file_stem = all_files[index].stem
    print(f"Close file {close_file_stem}")
    close_viewer = JupyterSegmentationViewer(close_file_stem, step_folder)
    dists_to_view = min_dists_for_each_face[index]
    close_viewer.display_faces_with_heatmap(dists_to_view, interval)

1
Close file 21242_6c2af7c2_7


0
Close file 21492_8bd34fc1_15


9
Close file 56436_2a8fc254_3


10
Close file 37117_89aac9d4_9


20
Close file 44647_d83249a9_0
